In [1]:
#Example of how to get Landsat data using the getImagesLib and view outputs using the Python visualization tools
#Acquires Landsat data and then adds them to the viewer
####################################################################################################
import os,sys
sys.path.append(os.getcwd())

#Module imports
import geeViz.getImagesLib as getImagesLib
ee = getImagesLib.ee
Map = getImagesLib.Map

print('done')

/Users/jessieeastburn/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Initializing GEE
Successfully initialized
geeViz package folder: /Users/jessieeastburn/Library/Python/3.9/lib/python/site-packages/geeViz
done


In [2]:
#pip install geeViz
#pip install geemap
#import geeViz as Map
import pandas as pd
import time
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

/var/folders/sf/r2b9cx3s6fl_d5hscrs0_44c0000gn/T/ipykernel_89656/2923351378.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:

# Get utka polygon.
studyArea = (
    ee.FeatureCollection('projects/disco-glass-413820/assets/plot_min_bound_geom')
    .filter("site_id == 'UOFU_UTKA'")
    .first()
    .geometry()
)

Map.port = 1990
Map.clearMap()

studyArea = ee.Geometry.Polygon(
            [[[-112.2647, 37.02876], 
              [-112.2647, 37.10894],
              [-112.0867, 37.10894],
              [-112.0867, 37.02876], 
              [-112.2647, 37.02876]]])
Map.centerObject(studyArea, 11)
Map.addLayer(studyArea, {'color': 'blue'}, 'utka')
Map.view()



Adding layer: utka
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:1990/geeView/ already serving.
cwd /Users/jessieeastburn/Library/CloudStorage/OneDrive-UniversityofUtah/URSA_Work/TS2/PJ_TimeSeries
geeView URL: http://localhost:1990/geeView/?projectID=None&accessToken=ya29.a0AfB_byAS7h8QNxX3YVeAq-z3rAnDeOCmN1Z40GdBIQpSoG2XjT0gtVCrGOZsewQdLeXHHxdheDLKqYKi1Lb0RwmdCAFZqHCumYZy3TfKuo-XGmgZ2yicoMVKiOTpIgOBEu2euLjq3fTqHARXG0oWE-wk13SlHI97VqhDMAaCgYKAegSARISFQHGX2MiQb67P2wTTsQZdeRwvYdSjw0173


::1 - - [20/Feb/2024 16:32:35] "GET /geeView/?projectID=None&accessToken=ya29.a0AfB_byAS7h8QNxX3YVeAq-z3rAnDeOCmN1Z40GdBIQpSoG2XjT0gtVCrGOZsewQdLeXHHxdheDLKqYKi1Lb0RwmdCAFZqHCumYZy3TfKuo-XGmgZ2yicoMVKiOTpIgOBEu2euLjq3fTqHARXG0oWE-wk13SlHI97VqhDMAaCgYKAegSARISFQHGX2MiQb67P2wTTsQZdeRwvYdSjw0173 HTTP/1.1" 200 -
::1 - - [20/Feb/2024 16:32:35] "GET /geeView/src/gee/gee-run/runGeeViz.js HTTP/1.1" 200 -


In [ ]:
studyArea

In [6]:
# Specify study area: Study area
# Can be a featureCollection, feature, or geometry
#studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary_buff2mile').geometry().bounds()#testAreas['CA']

# Update the startJulian and endJulian variables to indicate your seasonal 
# constraints. This supports wrapping for tropics and southern hemisphere.
# If using wrapping and the majority of the days occur in the second year, the system:time_start will default 
# to June 1 of that year.Otherwise, all system:time_starts will default to June 1 of the given year
# startJulian: Starting Julian date 
# endJulian: Ending Julian date
startJulian = 1
endJulian = 365

# Specify start and end years for all analyses
# More than a 3 year span should be provided for time series methods to work 
# well. If providing pre-computed stats for cloudScore and TDOM, this does not 
# matter
startYear = 2015
endYear = 2020

#Call on master wrapper function to get Landat scenes and composites
lsAndTs = getImagesLib.getLandsatWrapper(studyArea,startYear,endYear,startJulian,endJulian)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

# Indicate what type of image is being added to speed up map service creation
getImagesLib.vizParamsFalse['layerType']= 'geeImage';

Map.clearMap()
Map.port = 1990
#Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',True)
for year in range(startYear,endYear + 1 ):
     t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
     Map.addLayer(t.float(),getImagesLib.vizParamsFalse,'Default Params {} {}-{}'.format(year,startJulian,endJulian),'True')
#Map.centerObject(studyArea, 9)

Map.centerObject(studyArea, 11)
Map.addLayer(studyArea, {'color': 'blue'}, 'utka')

Map.turnOnInspector()
Map.view()


Get Processed Landsat: 
Start date: Jan 01 2015 , End date: Dec 30 2020
Applying scale factors for C2 L4 data
Applying scale factors for C2 L5 data
Applying scale factors for C2 L8 data
Only including SLC On Landsat 7
Applying scale factors for C2 L7 data
Applying scale factors for C2 L9 data
Applying Fmask Cloud Mask
Applying Fmask Shadow Mask
Adding layer: Default Params 2015 1-365
Adding layer: Default Params 2016 1-365
Adding layer: Default Params 2017 1-365
Adding layer: Default Params 2018 1-365
Adding layer: Default Params 2019 1-365
Adding layer: Default Params 2020 1-365
Adding layer: utka
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:1990/geeView/ already serving.
cwd /Users/jessieeastburn/Library/CloudStorage/OneDrive-UniversityofUtah/URSA_Work/TS2/PJ_TimeSeries
geeView URL: http://localhost:1990/geeView/?projectID=None&accessToken=ya29.a0AfB_byBpd0NUZCPiLkPV1kyamwmE_-WgzL8n4twK9Y4xm6aDmdYuyubOYj1_6z51aBH_6TM7ubckDD_eojwag8ShhC

::1 - - [20/Feb/2024 16:34:41] "GET /geeView/?projectID=None&accessToken=ya29.a0AfB_byBpd0NUZCPiLkPV1kyamwmE_-WgzL8n4twK9Y4xm6aDmdYuyubOYj1_6z51aBH_6TM7ubckDD_eojwag8ShhCBYe9_4ekESLuLS3rQRJ3E_XLgIBHQOBIXZNXFNy1ULqyFdDmFzmkrvzTCSTUxbqywd1p4nlziQfAaCgYKARISARISFQHGX2MiKDCjGYe4eH4RhUKhkOIy0Q0173 HTTP/1.1" 200 -
::1 - - [20/Feb/2024 16:34:41] "GET /geeView/src/gee/gee-run/runGeeViz.js HTTP/1.1" 200 -


In [ ]:
studyArea

In [ ]:
#You can access the parameters that were used through the properties of the returned collection
print(processedComposites.toDictionary().getInfo())
print(processedScenes.toDictionary().getInfo())

In [ ]:
#Since there are not that many images available in this area for these years, let's try adding Landsat 7
includeSLCOffL7 = True
#Call on master wrapper function to get Landat scenes and composites
lsAndTs = getImagesLib.getLandsatWrapper(studyArea,startYear,endYear,startJulian,endJulian,includeSLCOffL7=includeSLCOffL7)

#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

Map.clearMap()
#Turn off layers from previous iteration
Map.turnOffAllLayers()

# Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',False)
for year in range(startYear,endYear + 1 ):
     t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
     Map.addLayer(t.float(),getImagesLib.vizParamsFalse,'L7 added {} {}-{}'.format(year,startJulian,endJulian),'True')


Map.view()

#You'll notice this helps fill in the holes, but introduces many cloud-related artifacts

In [ ]:
#Let's try to improve the cloud masking. Fmask is used by default, but misses some clouds
#We'll try the adding in the cloudScore method
applyCloudScore = True

#Call on master wrapper function to get Landat scenes and composites
lsAndTs = getImagesLib.getLandsatWrapper(studyArea,startYear,endYear,startJulian,endJulian,includeSLCOffL7=includeSLCOffL7,applyCloudScore=applyCloudScore)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

#clear map
Map.clearMap()

#Turn off layers from previous iteration
Map.turnOffAllLayers()

Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'true'},'Time Series (NBR and NDVI)',True)
for year in range(startYear,endYear + 1 ):
     t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
     Map.addLayer(t.float(),getImagesLib.vizParamsFalse,'L7 and CloudScore added {} {}-{}'.format(year,startJulian,endJulian),'True')


Map.view()

#this cleans up the cloud masking a lot

In [ ]:
#You'll still notice there are some dark areas likely due to cloud shadow masking omission
#Fmasks's cloud shadow mask misses a lot typically. A temporal outlier method called the 
#Temporal Dark Outlier Mask (TDOM) works well with masking cloud shadows

#We'll try the cloudScore method
applyTDOM = True

#Call on master wrapper function to get Landat scenes and composites
#In order to identify dark outliers, we will extend the dates by 6 years to get a larger sample
lsAndTs = getImagesLib.getLandsatWrapper(studyArea,startYear-3,endYear+3,startJulian,endJulian,includeSLCOffL7=includeSLCOffL7,applyCloudScore=applyCloudScore,applyTDOM=applyTDOM)


#Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

#Turn off layers from previous iteration
Map.turnOffAllLayers()

Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',True)
for year in range(startYear,endYear + 1 ):
     t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
     Map.addLayer(t.float(),getImagesLib.vizParamsFalse,'CloudScore and TDOM added {} {}-{}'.format(year,startJulian,endJulian),'True')


Map.view()

#You'll notice this cleans up the cloud masking a lot

In [ ]:
# Run get images, set parameters manually, and export
lsAndTs = getImagesLib.getLandsatWrapper(\

    # Study area and time
    studyArea = studyArea,
    startYear = 2015,
    endYear = 2020,
    startJulian = 1,
    endJulian = 365,

    # Moving window options
    timebuffer =  0,# If multi-year moving window is needed, use timebuffer > 0 (e.g. timebuffer = 1 will result in a 3 year moving window)
    weights =  [1],# To compliment the timebuffer, set weights for each year of the moving window. This enables the center year to have more weight (e.g. [1,3,1] would weight the center year if timebuffer = 1 3x over the buffer years)

    # Composite methods and input imagery
    compositingMethod = 'medoid', # Specify median or medoid. Medoid allows for storing only the spectral and ancillary bands without indices
    toaOrSR = 'SR', # It is advised to only use TOA when combining Landsat and Sentinel-2 since the Sentinel-2 SR data in GEE are terrain-corrected and Landsat are not
    includeSLCOffL7 = False, # Whether to include Landsat 7 after May 2003. Scanline artifacts can be found in composites when True
    defringeL5 = True, # Whether to get rid of fringe edges found mostly on Landsat 4 and 5
    landsatCollectionVersion = 'C2', # To use Collection 1 or 2. Only use C1 if doing legacy work prior to 2022

    # Cloud and shadow masking
    # There will be a parameter for applying CloudScore+ to Sentinel-2 provided once it is available for all S2 data
    applyCloudScore = True, # We use this since fMask struggles for cloud masking
    applyTDOM = True, # We use TDOM for Landsat since the fMask cloud shadow masking method struggles a lot
    applyFmaskCloudMask = True,
    applyFmaskCloudShadowMask = True,
    applyFmaskSnowMask = False,
    cloudScoreThresh = 10,# The threshold applied to the cloudScore for Landsat and/or Sentinel-2
    performCloudScoreOffset = False,# Set this to False in areas with limited data and/or limited bright cool surfaces
    cloudScorePctl = 10, # If performCloudScoreOffset is set to True, this is the percentile of the cloudScores used to see if an area typically has a high cloudScore
    zScoreThresh = -1, # The TDOM Z-score threshold for finding dark outliers
    shadowSumThresh = 0.35,# The sum of the shadowSumBands has to be below this to be considered dark
    contractPixels = 1.5,# Inward buffer pixel radius to get rid of salt and pepper
    dilatePixels = 3.5, # Outwoard buffer pixel radius to dilate the cloud shadow mask by

    # Image processing
    correctIllumination = False, # Method for illuminating hill shadows. This is largely deprecated and not used
    correctScale = 250,

    # Export parameters
   # exportComposites = False,# Whether to export resulting composites exportCompositeCollection
   # exportPathRoot = 'projects/disco-glass-413820/assets/users/lcms-training_module-2_composites', #'content/drive/MyDrive/lcms-training_module-2_composites',used this i think #'projects/resolute-future-412019/assets/users/lcms-training_module-2_composites', og directory #'projects/earthengine-public/assets/content/drive/MyDrive/lcms-training_module-2_composites',
    exportComposites = True
    outputName = 'Landsat-UTKA',
    
    crs = 'EPSG:4326',
    #transform = getImagesLib.common_projections['NLCD_CONUS']['transform'],
    transform = None)
 #   scale = None,
  #  overwrite = True,# Whether to overwrite existing composites

    #Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

#Turn off layers from previous iteration
Map.turnOffAllLayers()

Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',True)
for year in range(startYear,endYear + 1 ):
     t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
     Map.addLayer(t.float(),getImagesLib.vizParamsFalse,'CloudScore and TDOM added {} {}-{}'.format(year,startJulian,endJulian),'True')

Map.turnOnInspector()
Map.view()
print('Done')

In [ ]:

#Turn off layers from previous iteration
Map.turnOffAllLayers()

Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',True)
for year in range(startYear,endYear + 1 ):
     t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
     Map.addLayer(t.float(),getImagesLib.vizParamsFalse,'CloudScore and TDOM added {} {}-{}'.format(year,startJulian,endJulian),'True')

Map.turnOnInspector()
Map.view()
print('Done')

In [ ]:
#Use this
#  imageForExport = processedComposites.first.clip(roi).unmask(outputNoData,False)
# Load an image
image = ee.Image(processedComposites.first())

# Define the region of interest (ROI)
roi = studyArea

# Define export parameters
export_params = {
    'image': image,
    'description': 'composite2015',
    'folder': 'Composites0220',
    'fileNamePrefix': 'composite2015',
    'scale': 30,
    'region': roi,
    'fileFormat': 'GeoTIFF'
}

In [ ]:
roi = studyArea
#Make sure image is clipped to roi in case it's a multi-part polygon
#imageForExport = processedComposites.clip(roi).unmask(outputNoData,False)
clipped_collection = processedComposites.map(lambda image: image.clip(studyArea))
clipped_collection

In [ ]:
def exportToDriveWrapper(imageForExport,outputName,driveFolderName,roi,scale= None,crs = 4326,transform = None,outputNoData = -32768):
  outputName = outputName.replace("/\s+/g",'-')#Get rid of any spaces

  #Pull geometry if feature or featureCollection
  try:
    roi = studyArea
  except Exception as e:
    x = e

  #Make sure image is clipped to roi in case it's a multi-part polygon
  imageForExport = clipped_collection.first().unmask(outputNoData,False)

  if transform != None and (str(type(transform)) == "<type 'list'>" or str(type(transform)) == "<class 'list'>"):
    transform = str(transform)


  #Ensure bounds are in export projection
  outRegion = roi.bounds(100,crs)

  # Map.addLayer(imageForExport,{},outputName,False)
  #t = ee.batch.Export.image.toDrive(imageForExport, outputName, driveFolderName, outputName, None, outRegion, scale, crs, transform, 1e13)
  #print('Exporting:',outputName)
  #t.start()

  t = ee.batch.Export.image.toDrive(imageForExport, outputName, driveFolderName, outputName, None, outRegion, scale, crs, transform, 1e13)
  print('Exporting:',outputName)
  t.start()

In [ ]:

Map.port = 1986
Map.clearMap()

#Turn off layers from previous iteration
Map.turnOffAllLayers()

Map.addLayer(utka_composites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',True)
for year in range(startYear,endYear + 1 ):
     t = utka_composites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
     Map.addLayer(t.float(),getImagesLib.vizParamsFalse,'CloudScore + TDOM {} {}-{}'.format(year,startJulian,endJulian),'True')
Map.centerObject(studyArea, 10)
Map.addLayer(studyArea, {'color': 'blue'}, 'UTKA Boundary')
Map.turnOnInspector()
Map.view()

In [7]:
# Run get images, set parameters manually, and export
lsAndTs = getImagesLib.getLandsatWrapper(\

    # Study area and time
    studyArea = studyArea,
    startYear = 2015,
    endYear = 2020,
    startJulian = 1,
    endJulian = 365,

    # Moving window options
    timebuffer =  0,# If multi-year moving window is needed, use timebuffer > 0 (e.g. timebuffer = 1 will result in a 3 year moving window)
    weights =  [1],# To compliment the timebuffer, set weights for each year of the moving window. This enables the center year to have more weight (e.g. [1,3,1] would weight the center year if timebuffer = 1 3x over the buffer years)

    # Composite methods and input imagery
    compositingMethod = 'medoid', # Specify median or medoid. Medoid allows for storing only the spectral and ancillary bands without indices
    toaOrSR = 'SR', # It is advised to only use TOA when combining Landsat and Sentinel-2 since the Sentinel-2 SR data in GEE are terrain-corrected and Landsat are not
    includeSLCOffL7 = False, # Whether to include Landsat 7 after May 2003. Scanline artifacts can be found in composites when True
    defringeL5 = True, # Whether to get rid of fringe edges found mostly on Landsat 4 and 5
    landsatCollectionVersion = 'C2', # To use Collection 1 or 2. Only use C1 if doing legacy work prior to 2022

    # Cloud and shadow masking
    # There will be a parameter for applying CloudScore+ to Sentinel-2 provided once it is available for all S2 data
    applyCloudScore = True, # We use this since fMask struggles for cloud masking
    applyTDOM = True, # We use TDOM for Landsat since the fMask cloud shadow masking method struggles a lot
    applyFmaskCloudMask = True,
    applyFmaskCloudShadowMask = True,
    applyFmaskSnowMask = False,
    cloudScoreThresh = 10,# The threshold applied to the cloudScore for Landsat and/or Sentinel-2
    performCloudScoreOffset = False,# Set this to False in areas with limited data and/or limited bright cool surfaces
    cloudScorePctl = 10, # If performCloudScoreOffset is set to True, this is the percentile of the cloudScores used to see if an area typically has a high cloudScore
    zScoreThresh = -1, # The TDOM Z-score threshold for finding dark outliers
    shadowSumThresh = 0.35,# The sum of the shadowSumBands has to be below this to be considered dark
    contractPixels = 1.5,# Inward buffer pixel radius to get rid of salt and pepper
    dilatePixels = 3.5, # Outwoard buffer pixel radius to dilate the cloud shadow mask by

    # Image processing
    correctIllumination = False, # Method for illuminating hill shadows. This is largely deprecated and not used
    correctScale = 250,

    # Export parameters
   # exportComposites = False,# Whether to export resulting composites exportCompositeCollection
   # exportPathRoot = 'projects/disco-glass-413820/assets/users/lcms-training_module-2_composites', #'content/drive/MyDrive/lcms-training_module-2_composites',used this i think #'projects/resolute-future-412019/assets/users/lcms-training_module-2_composites', og directory #'projects/earthengine-public/assets/content/drive/MyDrive/lcms-training_module-2_composites',
    exportComposites = False,
    #outputName = 'Landsat-UTKA',
    
    crs = 'EPSG:4326',
    #transform = getImagesLib.common_projections['NLCD_CONUS']['transform'],
    transform = None)
 #  scale = None,
  #  overwrite = True,# Whether to overwrite existing composites

    #Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

#clear map
Map.clearMap()
#Turn off layers from previous iteration
Map.turnOffAllLayers()

#Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',True)
for year in range(startYear,endYear + 1 ):
     t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
     Map.addLayer(t.float(),getImagesLib.vizParamsFalse,'CS and TDOM UTKA {} {}-{}'.format(year,startJulian,endJulian),'True')

Map.turnOnInspector()
Map.view()
print('Done')

Get Processed Landsat: 
Start date: Jan 01 2015 , End date: Dec 30 2020
Applying scale factors for C2 L4 data
Applying scale factors for C2 L5 data
Defringing L4 and L5
Applying scale factors for C2 L8 data
Only including SLC On Landsat 7
Applying scale factors for C2 L7 data
Applying scale factors for C2 L9 data
Applying Cloud Score
Not computing cloudScore offset
Applying Fmask Cloud Mask
Applying TDOM Shadow Mask
Computing irMean for TDOM
Computing irStdDev for TDOM
Applying Fmask Shadow Mask
Adding layer: CS and TDOM UTKA 2015 1-365
Adding layer: CS and TDOM UTKA 2016 1-365
Adding layer: CS and TDOM UTKA 2017 1-365
Adding layer: CS and TDOM UTKA 2018 1-365
Adding layer: CS and TDOM UTKA 2019 1-365
Adding layer: CS and TDOM UTKA 2020 1-365
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:1990/geeView/ already serving.
cwd /Users/jessieeastburn/Library/CloudStorage/OneDrive-UniversityofUtah/URSA_Work/TS2/PJ_TimeSeries
geeView URL: http://l

Done


::1 - - [20/Feb/2024 16:34:53] "GET /geeView/?projectID=None&accessToken=ya29.a0AfB_byBfJfDlRYVzEEQuvzafg72lQUvvVGTL7IeWNeW5A2uDAEPdFV7bIRXDn64DrtoZwhTlJY3CKcCN4VJQPYUowMYNwmtFmHBi093kq90EYRzB9hxkB_ar07mL-6lbgKRwDO7fTka2fx215Ogg3gEgfH00L2gQYksdGQaCgYKAWQSARISFQHGX2Mi1mpaxn8NXyG3mk3eiGJosg0173 HTTP/1.1" 200 -
::1 - - [20/Feb/2024 16:34:53] "GET /geeView/src/gee/gee-run/runGeeViz.js HTTP/1.1" 200 -


In [8]:
clipped_collection = processedComposites.map(lambda image: image.clip(studyArea))
image1 = clipped_collection.first()

In [13]:
#USed to export images. 

def exportToDriveWrapper(imageForExport,outputName,driveFolderName,roi,scale= None,crs = 4326,transform = None,outputNoData = -32768):
  outputName = outputName.replace("/\s+/g",'-')#Get rid of any spaces

  #Pull geometry if feature or featureCollection
  try:
    roi = studyArea
  except Exception as e:
    x = e

  #Make sure image is clipped to roi in case it's a multi-part polygon
  imageForExport = clipped_collection.first().unmask(outputNoData,False)
  outputName = 'utka2015'
  driveFolderName = 'Composites0220'

  if transform != None and (str(type(transform)) == "<type 'list'>" or str(type(transform)) == "<class 'list'>"):
    transform = str(transform)


  #Ensure bounds are in export projection
  outRegion = roi.bounds(100,crs)

  # Map.addLayer(imageForExport,{},outputName,False)
  #t = ee.batch.Export.image.toDrive(imageForExport, outputName, driveFolderName, outputName, None, outRegion, scale, crs, transform, 1e13)
  #print('Exporting:',outputName)
  #t.start()

  t = ee.batch.Export.image.toDrive(imageForExport, outputName, driveFolderName, outputName, None, outRegion, scale, crs, transform, 1e13)
  print('Exporting:',outputName)
  t.start()

In [ ]:
#Run for each image

roi = studyArea

def exportToDriveWrapper(imageForExport, outputName, driveFolderName, roi, scale=None, crs='EPSG:4326', transform=None, outputNoData=-32768):
    outputName = outputName.replace("/\s+/g", '-')  # Get rid of any spaces

    # Ensure bounds are in export projection
    outRegion = roi.bounds(100, crs)

    # Export the image to Google Drive
    t = ee.batch.Export.image.toDrive(imageForExport, outputName, driveFolderName, outputName, None, outRegion, scale, crs, transform, 1e13)
    print('Exporting:', outputName)
    t.start()

# Example usage
for image in clipped_collection.getInfo()['features']:
    image_id = image['year']
    output_name = image_id.replace("/", "_")
    exportToDriveWrapper(ee.Image(image_id), output_name, 'Composites0220', roi, scale=10, crs='EPSG:4326', transform=None)


In [ ]:
clipped_collection.getInfo()['features']

In [ ]:
clipped_collection = processedComposites.map(lambda image: image.clip(studyArea))


def exportToDriveWrapper(imageForExport,outputName,driveFolderName,roi,scale= None,crs = 4326,transform = None,outputNoData = -32768):
  outputName = outputName.replace("/\s+/g",'-')#Get rid of any spaces

  #Pull geometry if feature or featureCollection
  try:
    roi = studyArea
  except Exception as e:
    x = e

  #Make sure image is clipped to roi in case it's a multi-part polygon
  imageForExport = clipped_collection.first().unmask(outputNoData,False)

  if transform != None and (str(type(transform)) == "<type 'list'>" or str(type(transform)) == "<class 'list'>"):
    transform = str(transform)


  #Ensure bounds are in export projection
  outRegion = roi.bounds(100,crs)

  # Map.addLayer(imageForExport,{},outputName,False)
  #t = ee.batch.Export.image.toDrive(imageForExport, outputName, driveFolderName, outputName, None, outRegion, scale, crs, transform, 1e13)
  #print('Exporting:',outputName)
  #t.start()

  t = ee.batch.Export.image.toDrive(imageForExport, outputName, driveFolderName, outputName, None, outRegion, scale, crs, transform, 1e13)
  print('Exporting:',outputName)
  t.start()